In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [1]:
import pandas as pd
import h5py

rawdata = pd.read_csv("data/unarXive.clear/query.question.results.good_questions.csv")

sectionsf = h5py.File('data/unarXive.clear/unarXive.clear.sections.h5', 'r')

from torch.utils.data import Dataset
from uniem.data_structures import RecordType, PairRecord, TripletRecord, ScoredPairRecord
class UnarXive_Question_Sentense_Dataset(Dataset):
    def __init__(self, rawdata):
        self.rawdata      = rawdata
    
    def __getitem__(self, idx):
        paper_id, sentense_id , question = rawdata.iloc[32][['paper_id','sentense_id','question']]
        evidence = sectionsf.get(f'{paper_id}/{sentense_id}')[()].decode('utf-8')
        return dict(text=question, text_pos=evidence)

dataset = UnarXive_Question_Sentense_Dataset(rawdata)



from uniem.finetuner import FineTuner
# 指定训练的模型为 m3e-small
finetuner = FineTuner.from_pretrained('/home/zhangtianning/.cache/torch/sentence_transformers/moka-ai_m3e-base', dataset=dataset)
fintuned_model = finetuner.run(epochs=30, output_dir='finetuned-model-riddle',batch_size=30)